As a undergrade student (I have been studying Statistcs) I have a huge interest in Machin Learning or Statistical modeling and in order to improve my skills, I have made this notebook to record some scripts of mine - I've done it in Python, but it could have done in R as well.
The dataset was provided by Kaggle that is a web page for machine learning challanges - which could be find here : https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

At first, I have started by importing the pandas "package" for reading the dataset

In [2]:
import pandas as pd
import os

os.getcwd()
os.chdir('C:/Users/swl5565/Documents/Bases')
df = pd.read_csv('train.csv')

By checking the first five row in our dataset we can see an Id and Comment_Text columns, which one has our features or variables that would feed our model. The others columns are our target - our goal is to predict those columns given the comment_text feature

In [3]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [134]:
df.shape # prints the dataset dimensions (rows x columns)

(159571, 8)

It is a good practice for looking for missings data and, since we don't have it, we are good to gol!

In [4]:
df.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

Since we have to work with words as features, we need to count their frequencies and also use them to feed our future model.
The Scikit-Learn module has a good function for this task.

In [73]:
from sklearn.feature_extraction.text import CountVectorizer

cvec = CountVectorizer(stop_words='english', min_df=.0025, max_df=.1, ngram_range = (1, 6))

In [86]:
# Calculate all the n-grdfs found in all documents
from itertools import islice

cvec.fit(df['comment_text'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.1, max_features=None, min_df=0.0025,
        ngram_range=(1, 6), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

Let's check some words in our dictionary!

In [87]:
list(islice(cvec.vocabulary_.items(), 10))

[('explanation', 707),
 ('edits', 634),
 ('username', 2017),
 ('fan', 734),
 ('reverted', 1632),
 ('weren', 2110),
 ('new', 1270),
 ('york', 2201),
 ('remove', 1582),
 ('template', 1889)]

The next step is to check the how spare our matrix (dictionary)

In [88]:
X_counts = cvec.transform(df['comment_text'])

print('sparse matrix shape:', X_counts.shape)
print('nonzero count:', X_counts.nnz)
print('sparsity: %.2f%%' % (100.0 * X_counts.nnz / (X_counts.shape[0] * X_counts.shape[1])))

sparse matrix shape: (159571, 2205)
nonzero count: 2729563
sparsity: 0.78%


I am not an text miner expert, but since our matrix has just 0,83% of sparsity I will not use the TfidfTransformer

Once we have our dictionary which means words and their frequencies, saving them into a dataframe could be useful for future exploratory analysis ( barplot, boxplot and etc) which one I will not do - my focus here with this notebook is just to fit a model, but the next code line would create that dataframe

In [90]:
occ = np.asarray(X_counts.sum(axis=0)).ravel().tolist()
counts_df = pd.DataFrame({'term': cvec.get_feature_names(), 'occurrences': occ})
counts_df.sort_values(by='occurrences', ascending=False).head(20)

,occurrences,term
1047,19144,know
1385,18468,people
614,18288,edit
191,16712,articles
2002,16594,use
1935,15885,time
555,14953,did
2013,14101,user
1915,13904,thanks
1873,13430,talk page


Let's begin the modelling part.
For the first moment I'll import some useful modules provided by Scikit-Learn

In [102]:
from sklearn.cross_validation import train_test_split, cross_val_predict #provide an train /test split on the data set
from sklearn.tree import DecisionTreeClassifier # code for importing the Decision Tree model
from sklearn.ensemble import RandomForestClassifier # code for importing the Random Forest model

In this notebook I'll face the Decision Tree against the Random Forest model (which could be considered an "evolution" of the other one)

In [103]:
y = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']] # Define the target variable or feature

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X_counts, y, test_size = .3) # split our dataset into train/test

With the cross_val_predict function it's possible to use cross validation for finding the "best" (considering just the Decision Tree or the Random Forest).
Obs : Use the cross validation could be a powerful tool to avoid the overfitting!

In [112]:
#Code for cross validation or K-Fold, where K = 10
y_pred_cv_tree = cross_val_predict(DecisionTreeClassifier(), X_train, y_train, cv = 10)
y_pred_cv_rf = cross_val_predict(RandomForestClassifier(), X_train, y_train, cv = 10)

At this point, we should examine the models' predictive capacity

In [108]:
from sklearn.metrics import classification_report # code

In [113]:
print('\nCross Validation\n\nDecision Tree: \n\n',
     classification_report(y_train, y_pred_cv_tree),
     '\n\nRandom Forest:\n\n', classification_report(y_train, y_pred_cv_rf)
     )


Cross Validation

Decision Tree: 

              precision    recall  f1-score   support

          0       0.51      0.59      0.55     10759
          1       0.29      0.30      0.29      1118
          2       0.62      0.67      0.64      5945
          3       0.19      0.17      0.18       339
          4       0.48      0.53      0.51      5544
          5       0.21      0.20      0.20       990

avg / total       0.50      0.56      0.53     24695
 

Random Forest:

              precision    recall  f1-score   support

          0       0.73      0.57      0.64     10759
          1       0.41      0.22      0.28      1118
          2       0.80      0.65      0.71      5945
          3       0.44      0.10      0.16       339
          4       0.67      0.51      0.58      5544
          5       0.36      0.11      0.17       990

avg / total       0.70      0.53      0.60     24695



It looks like the Random Forest have performed better than the Decision Tree - actually that behavior was expected and you could check this result by reading the book The Elements of Statistical Learning which was written by Trevor Hastie, Robert Tibshirani and Jerome H. Friedman

Now that we have done with the cross validation, let's fit our model and then use it to predict the sample test.
Obs : I could have done more with cross validation in order to improve the Random Forest (e.g, grid_search, permutation_test, under/oversampling and etc), however my personal computer is not good enougth for this dataset whose len is equal to 159571 rows per 8 columns

In [128]:
final_model = RandomForestClassifier(oob_score = True).fit(X_train, y_train)

C:\Users\swl5565\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\swl5565\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [131]:
print('\nModel: Random Forest\n\n Sample Test Predictions\n',
     classification_report(y_test, final_model.predict(X_test))
     )


Model: Random Forest

 Sample Test Predictions
              precision    recall  f1-score   support

          0       0.73      0.56      0.64      4535
          1       0.40      0.21      0.27       477
          2       0.80      0.65      0.72      2504
          3       0.53      0.12      0.19       139
          4       0.67      0.50      0.57      2333
          5       0.35      0.12      0.18       415

avg / total       0.70      0.53      0.60     10403



Obs : The labels 0, 1, 2, ..., 5 at the classification report means 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'
    